Data Visualization Lab 3

In [ ]:
pip install geopandas

Question 1 (Bubble map to show the populations of all the state capital of India)

In [48]:
from urllib import request
# import required libraries
import pandas as pd
import numpy as numpy
import folium
from shapely.geometry import Point
import geopandas as gpd
import webbrowser
import matplotlib.pyplot as plt
import requests

In [ ]:
# Import dataset
df=pd.read_csv("/lab3 - Sheet1.csv")
df2=df.copy()
# Using folium library create bubble map of India
contry_bubble_map=folium.Map(location=[21.53, 78.69], zoom_start=5.5)
for i, j in df.iterrows():
    folium.CircleMarker([j['latitude'], j['longitude']],radius=j['population']/500000,fill_color='skyblue',color='blue',fill=True,).add_child(folium.Popup(j['state'] + ": " + str(j['population']) + " people")).add_to(contry_bubble_map)

# Show bubble map
contry_bubble_map


In [ ]:
data_=gpd.GeoDataFrame(df2,geometry=[Point(xy) for xy in zip(df2.longitude,df2.latitude)])
map_bubble_india=folium.Map(location=[21.53, 78.69],zoom_start=5.5)

# Bubble map of Indai using shapely and folium
data_.apply(lambda x: folium.CircleMarker(location=[x.geometry.y, x.geometry.x],radius=x.population/500000,
                                        color='skyblue',fill_color='blue',fill=True,
                                       ).add_child(folium.Tooltip(f"Latitude: {x.latitude},Longitude:{x.longitude},Population:{x.population}"))
                                        .add_to(map_bubble_india),axis=1)
# Show bubble map
map_bubble_india

Question 2 ( Connection graph of Go Air domestic flights of India)

In [ ]:
# Load Given dataset
df=pd.read_csv("/flight_data.csv")
print(df.head())
df.describe()

In [ ]:
# Unique Values
origin1=pd.unique(df['Origin'])
dest1 = pd.unique(df['Destination'])
cities=numpy.unique(numpy.concatenate((origin1,dest1),0))
cities

In [71]:
latitudes=[]
longitudes=[]
API_KEY='249958e2b520111a8a49669c9f43b007'
for i in cities:
   response = requests.get(f'https://nominatim.openstreetmap.org/search?q={i}&format=json')
   json_data=response.json()
   lon=(json_data[0]['lon'])
   lat=(json_data[0]['lat'])
   latitudes.append(lat)
   longitudes.append(lon)



In [73]:
coord_lat={k:val for k,val in zip(cities, latitudes)}
coord_lon={k:val for k,val in zip(cities, longitudes)}

In [74]:
df['origin_lat']=df['Origin'].map(coord_lat)
df['origin_lon']=df['Origin'].map(coord_lon)
df['destination_lat']=df['Destination'].map(coord_lat)
df['destination_lon']=df['Destination'].map(coord_lon)


In [ ]:
df['origin_count']=df['Origin'].map(df['Origin'].value_counts(dropna=False).to_dict())
df

In [78]:
df.to_csv("flight_with_coordinates.csv", index=False)

In [ ]:
df2=pd.read_csv("./flight_with_coordinates.csv")
df2.head()

In [ ]:
# Connection Map using Folium
connection_map=folium.Map(location=[21.53, 78.69], zoom_start=5.5)
for i, j in df2.iterrows():
    folium.CircleMarker(location=[j['origin_lat'], j['origin_lon']],
                        radius=(j['origin_count']/10),color='blue',
                        fill=True,fill_color=None,).add_child(folium.Tooltip(f"Place: {j['Origin']}, Longitude: {j['origin_lon']}, Latitude: {j['origin_lat']} \n Origin Count: {origin_count}")).add_to(connection_map)
    
    folium.PolyLine([[j['origin_lat'], j['origin_lon']], [j['destination_lat'], j['destination_lon']]], 
                  color='blue',opacity=0.2
                ).add_child(folium.Tooltip(f"Route: {j['Origin']} to {j['Destination']} \n Routing: {j['Routing']} \n Flight No: {j['Flight Number']}")).add_to(connection_map)
# Show connection map
connection_map